In [ ]:
import os
import numpy as np
import cv2
import pywt

"""
We use pywt to perform DWT-based steganography.
"""
def dwt_embed(cover_image_path, secret_image_path):
    cover_image = cv2.imread(cover_image_path, cv2.IMREAD_GRAYSCALE)
    secret_image = cv2.imread(secret_image_path, cv2.IMREAD_GRAYSCALE)

    cover_coeffs = pywt.dwt2(cover_image, 'haar')
    secret_coeffs = pywt.dwt2(secret_image, 'haar')
    
    # embedding here
    stego_coeffs = (cover_coeffs[0] + secret_coeffs[0], cover_coeffs[1])

    stego_image = pywt.idwt2(stego_coeffs, 'haar').clip(0, 255).astype(np.uint8)
    return stego_image

# Paths to the directories containing the cover and hidden images
cover_path = './test/cover/class1'
hidden_path = './test/hidden/class1'

# Path to the directory where the resulting stego images will be saved
stego_path = './test/stego/class1'

# Loop through all files in the cover directory
for i, cover_filename in enumerate(os.listdir(cover_path)):
    if cover_filename.endswith('.jpg') or cover_filename.endswith('.jpeg') or cover_filename.endswith('.png'):
        # Get the path to the cover image
        cover_image_path = os.path.join(cover_path, cover_filename)

        # Get the path to the corresponding secret image based on the index
        hidden_filename = os.listdir(hidden_path)[i]
        if hidden_filename.endswith('.jpg') or hidden_filename.endswith('.jpeg') or hidden_filename.endswith('.png'):
            hidden_image_path = os.path.join(hidden_path, hidden_filename)

            # Embed the secret image into the cover image
            stego_image = dwt_embed(cover_image_path, hidden_image_path)

            # Save the stego image into the stego directory with a new name
            stego_filename = f'{cover_filename.split(".")[0]}_{hidden_filename.split(".")[0]}.jpg'
            stego_image_path = os.path.join(stego_path, stego_filename)
            cv2.imwrite(stego_image_path, stego_image)


In [ ]:
import os
import cv2
import numpy as np
'''
The function helps to generate original stego files, and since the extraction is lossy, we need to make the lossy hidden images 
as an input for our model.
'''
def PVD_embed(cover_image, secret_image):

    height, width = cover_image.shape
    stego_img = np.zeros((height, width), dtype=np.uint8)

    for i in range(height):
        for j in range(width):
            # Use int to avoid unsafe float operations, but it will lead to loosy of data
            cover_pixel = int(cover_image[i, j])
            secret_pixel = int(secret_image[i, j])
            # Embed secret_pixel into cover_pixel using PVD
            stego_pixel = (cover_pixel & 0xFE) | (secret_pixel >> 7)
            stego_img[i, j] = stego_pixel

    return stego_img

def PVD_extract(stego_image):

    height, width = stego_image.shape
    extracted_img = np.zeros((height, width), dtype=np.uint8)

    for i in range(height):
        for j in range(width):
            stego_pixel = int(stego_image[i, j])
            # Extract secret_pixel from stego_pixel using PVD
            secret_pixel = ((stego_pixel & 0x01) << 7)
            extracted_img[i, j] = secret_pixel

    return extracted_img




In [ ]:
def embed_images_from_folders(cover_folder, secret_folder, output_folder):
    cover_images = sorted(os.listdir(cover_folder))
    secret_images = sorted(os.listdir(secret_folder))


    for cover_img_name, secret_img_name in zip(cover_images, secret_images):
        cover_path = os.path.join(cover_folder, cover_img_name)
        secret_path = os.path.join(secret_folder, secret_img_name)

        cover_image = cv2.imread(cover_path, cv2.IMREAD_GRAYSCALE)
        secret_image = cv2.imread(secret_path, cv2.IMREAD_GRAYSCALE)

        stego_img = PVD_embed(cover_image, secret_image)
        output_path = os.path.join(output_folder, f'{cover_img_name.split(".")[0]}_{secret_img_name.split(".")[0]}.png')
        cv2.imwrite(output_path, stego_img)

cover_folder = './train/cover/class1'
secret_folder = './train/hidden/class1'
output_folder = './train/stego/class1'
"""
Generate stego images for train test and valid
Remember to remove .ipynb_checkpoints using rm -r .ipynb_checkpoints
"""
embed_images_from_folders(cover_folder, secret_folder, output_folder)

In [ ]:
def extract_images_from_folders(stego_folder, output_folder):
    stego_images = sorted(os.listdir(stego_folder))

    for stego_img_name in stego_images:
        stego_path = os.path.join(stego_folder, stego_img_name)

        stego_image = cv2.imread(stego_path, cv2.IMREAD_GRAYSCALE)

        extract_img = PVD_extract(stego_image)
        output_path = os.path.join(output_folder, f'extrac_{stego_img_name}.png')
        cv2.imwrite(output_path, extract_img)
"""
Need to generated hidden images for PVD, saved into PVD_hidden
Also do this for train test and valid
Remember to remove .ipynb_checkpoints using rm -r .ipynb_checkpoints
"""

stego_folder = './train/PVD_stego/class1'
output_folder = './train/PVD_hidden/class1'

extract_images_from_folders(stego_folder, output_folder)
